In [37]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import *
from nltk.stem.porter import *
import os
from Constants import *
import pickle
import gensim 
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [38]:
# ! Uncomment in first run 
# nltk.download('punkt')
# nltk.download('words')
# nltk.download('stopwords')

In [39]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
root = Path(".")

In [40]:
my_path = root / "Pickled_files" / "Documents"
dbfile = open(my_path, 'rb')     
documents = pickle.load(dbfile)
dbfile.close()

In [41]:
sample = open("./alice.txt", "r",encoding="utf8") 
s = sample.read() 
f = s.replace("\n", " ") 

data = list()
for i in sent_tokenize(f): 
    temp = list()
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
    data.append(temp) 

In [42]:
cbow_model = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5)

In [43]:
def preprocess(text):
    tokens = []
    for word in word_tokenize(text, language='english'):
        if len(word) >= 2 and word not in stop_words:
            tokens.append(word)
    return tokens

In [44]:
preprocess(documents[0][0])

['ontario',
 'automobile',
 'policy',
 'oap',
 'owner',
 'policy',
 'approved',
 'superintendent',
 'financial',
 'services',
 'use',
 'standard',
 'owner',
 "'s",
 'policy',
 'june',
 '2016',
 'policy',
 'automobile',
 'insurance',
 'policy.it',
 'written',
 'easy',
 'understand',
 'language.please',
 'read',
 'carefully',
 'know',
 'rights',
 'obligations',
 'rights',
 'obligations',
 'insurance',
 'company']

In [45]:
# Define a function to generate text embeddings
def text_embedding(text, model):
    # Preprocess the text (tokenize, remove stop words, stem)
    preprocessed_text = preprocess(text)
    # Get the word embeddings for each word in the document
    word_embeddings = [model.wv[word] for word in preprocessed_text if word in model.wv.key_to_index]
    # Take the average of the word embeddings to generate a document embedding
    if len(word_embeddings) > 0:
        document_embedding = np.mean(word_embeddings, axis=0)
    else:
        document_embedding = np.zeros(model.vector_size)
    return document_embedding

In [46]:
# Define a function to calculate cosine similarity between two vectors
def cosine_similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [47]:
document_vecs = [text_embedding(document[0], cbow_model) for document in documents]

In [48]:
pd.DataFrame(document_vecs)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.036199,0.176172,0.043309,0.112466,0.031488,-0.249989,0.048573,0.351634,-0.062171,-0.198977,...,0.134122,0.083872,0.133441,-0.026218,0.280630,0.157938,0.069733,-0.015436,0.094636,-0.101888
1,-0.006597,0.071246,-0.010354,0.059451,0.040848,-0.128437,0.057911,0.175092,-0.043942,-0.097356,...,0.070992,0.019820,0.058979,0.020816,0.116368,0.059367,0.039245,-0.057019,0.059942,-0.029947
2,-0.014308,0.105204,-0.007701,0.082347,0.044618,-0.178956,0.070149,0.244710,-0.057172,-0.135477,...,0.096060,0.038473,0.085434,0.014027,0.171812,0.085685,0.051522,-0.060833,0.084581,-0.052760
3,-0.014345,0.075093,-0.009410,0.061449,0.036152,-0.134049,0.053145,0.181545,-0.042634,-0.102516,...,0.071698,0.028568,0.063889,0.014489,0.126703,0.065252,0.039935,-0.049451,0.061021,-0.035227
4,-0.009247,0.064044,-0.008323,0.052144,0.036680,-0.121774,0.053386,0.159615,-0.039609,-0.091186,...,0.064795,0.021182,0.053493,0.015847,0.111111,0.056983,0.035355,-0.050098,0.056131,-0.030071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373,-0.028998,0.164099,-0.007789,0.136320,0.056662,-0.278809,0.100352,0.396562,-0.087125,-0.211744,...,0.139868,0.061336,0.144839,0.017013,0.282420,0.149058,0.075946,-0.078662,0.120741,-0.072435
2374,-0.022491,0.129931,0.000708,0.096417,0.053951,-0.225333,0.084274,0.299282,-0.065933,-0.174147,...,0.117757,0.049648,0.107792,0.008898,0.216438,0.117646,0.065420,-0.063957,0.092226,-0.067804
2375,-0.038807,0.190425,-0.002074,0.148503,0.059547,-0.319044,0.112842,0.445374,-0.091780,-0.242606,...,0.162075,0.069339,0.160665,0.012951,0.322726,0.172941,0.084109,-0.074060,0.129034,-0.086018
2376,-0.056023,0.244131,0.048500,0.165973,0.015929,-0.362919,0.075294,0.488349,-0.077654,-0.236086,...,0.165115,0.080938,0.190347,-0.042741,0.372129,0.203564,0.057683,-0.025757,0.109795,-0.112150


In [49]:
my_path = root / "Pickled_files" / "Document_vectors"
dbfile = open(my_path, 'wb')
pickle.dump(document_vecs, dbfile) 
dbfile.close()

In [50]:
my_path = root / "Pickled_files" / "Document_vectors"
dbfile = open(my_path, 'rb')     
document_vecs = pickle.load(dbfile)
dbfile.close()

In [51]:
# Example usage: compare a query with a set of documents
query = "policy"

query_vec = text_embedding(query,cbow_model)
# Calculate similarity scores between the query and all documents
similarity_scores = [cosine_similarity(query_vec, document_vec) for document_vec in document_vecs]
# Rank documents based on similarity scores
ranked_documents = [document for _, document in sorted(zip(similarity_scores, documents), reverse=True)]
# Print the ranked documents
ranked_documents[0:5]

[('bear the expenses of the appraisal and um- pire equally.b.we do not waive any of our rights under this policy by agreeing to an appraisal . ',
  '.\\Docs\\Auto\\PL-600003-87.docx'),
 ("servicing ; 6.loss to a camper body or `` trailer '' you own transportation expenses which is not shown in the declarations.this exclusion ( 6 . ) ",
  '.\\Docs\\Auto\\PL-600003-87.docx'),
 ("loss to equipment designed for the member '' .however , `` non-owned auto '' does reproduction of sound.this exclusion ( 4 . ) ",
  '.\\Docs\\Auto\\PL-600003-87.docx'),
 ("breakage of glass.than 24 hours ; and 6.hail , water or the loss is caused by `` collision '' or covered flood ; under coverage f - comprehensive of if breakage of glass is caused by a `` collision '' this policy . ",
  '.\\Docs\\Auto\\PL-600003-87.docx'),
 ("these limits are the most we will pay regardless of the num- 1.pay the expenses it incurs ; and ber of : 2.bear the expenses of the third arbitrator '' insureds '' ; equally . ",
  '.\\Doc